In [3]:
from IPython import get_ipython 
from nbformat import read

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
import os 
path = os.path.dirname(os.path.realpath('ex2'))
import sys 
sys.path.insert (0, path) 
import searchEngine as se

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amparoalias/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import math

In [7]:
df_tweets = se.df_tweets
df_tweets.head(1)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,quoted_status_id,quoted_status_id_str,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status
0,Thu Dec 17 18:23:49 +0000 2020,1339637441238454273,1339637441238454273,I’ve seen a huge amount of people pleading to ...,False,"[0, 256]","{'hashtags': [{'text': 'BLM', 'indices': [252,...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,NaN,NaN,0,0,False,False,en,NaN,NaN,NaN


## Word2Vec

#### Training the model of word2vec

In [8]:
#!pip install gensim

In [9]:
from gensim.models import Word2Vec

In [10]:
model = Word2Vec(window=5, min_count=1, workers=4,size=10)

tweets_cleaned = [se.getTerms(text) for text in df_tweets["full_text"]]
model.build_vocab(tweets_cleaned)

In [11]:
model.train(tweets_cleaned,total_examples=len(tweets_cleaned),epochs=10)

(2289428, 2528250)

#### Transformation of tweets and the query

In [12]:
def tweet2vec(text,word2vec):
    for i,word in enumerate(text):
        #print(word)
    
        word_vector = word2vec[word]
        #print(word_vector.shape)
        if i  > 0:
            vectors = np.concatenate([vectors,word_vector])
        if i == 0:
            vectors = word_vector  
    return vectors.reshape(len(text),-1).mean(axis=0)


In [13]:
def query2vec(query, word2vec):
    query=se.getTerms(query)
    for i,word in enumerate(query):
    
        word_vector = word2vec[word]
        if i  > 0:
            vectors = np.concatenate([vectors,word_vector])
        if i == 0:
            vectors = word_vector  
    return vectors.reshape(len(query),-1).mean(axis=0)


In [14]:
representation = [tweet2vec(items,model) for items in tweets_cleaned]
embeded_tweets = np.array(representation)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


## TSNE

In [15]:
from sklearn.manifold import TSNE

In [16]:
tsne_model_en_2d = TSNE(perplexity=15, n_components=2, init='pca', n_iter=250, random_state=32)
embeddings_en_2d = np.array(tsne_model_en_2d.fit_transform(embeded_tweets))

## Clustering

In [17]:
CLUSTERS = 2

In [18]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=CLUSTERS, random_state=0,max_iter=1000).fit(embeddings_en_2d)
df_tweets["cluster"] = kmeans.labels_


## Score

In [19]:
def diversify_score(df_tweets,num_clusters=5):
    '''Returs the diversify score for a given set of tweets,
    input must be a df with the field of cluster defined'''
    #print(np.unique(df_tweets.cluster.values,return_counts=True))
    score = 0
    
    for i in range(5):
        cluster_items = len(df_tweets[df_tweets.cluster==i])
        if  cluster_items > 0:
            # For each cluster, the query can have a score up to 1.
            individual_score = 1 if cluster_items/(len(df_tweets)/num_clusters) > 1 else cluster_items/(len(df_tweets)/num_clusters)
            #print(individual_score)
            score = individual_score*(1/num_clusters) + score
            #print(df_tweets[df_tweets.cluster==i].shape[0])
    return score 

In [20]:
def diversify_correction(results,top):
    new_df = pd.DataFrame(columns = results.columns)
    residue,integer = math.modf(top/CLUSTERS)
    #print(math.modf(top/CLUSTERS))
    residue = np.round(residue/CLUSTERS)
    for i in range(int(integer)):
        for cluster in range(CLUSTERS):
            #print(results[results.cluster==cluster].shape)
            if results[results.cluster==cluster].shape[0] > i:
                new_df =new_df.append(results[results.cluster==cluster].iloc[i])
            else:
                pass
    for i in range(int(residue)):
         new_df =new_df.append(results[results.cluster==cluster].iloc[i])
    return new_df      

In [21]:
def userInteraction():
    print("Insert: \n1 if you want to use the inverted index \n2 if you want to use our score,\n3 if you want to return a more diverse output")
    scoring = input()
    scoring = int(scoring)

    print("\nHow many returned tweets you want?")
    
    top = input()
    top = int(top)

    print("\nInsert your query:")
    query_input = input()
    
    ranked_docs = se.search_tweets(query_input, se.index, scoring)      
    
    print("\n======================\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
    results = pd.DataFrame(columns=['for_query','tweet', 'username', 'date', 'hashtags', 'likes', 'retweets', 'url','cluster'])
    count = 0
    
    for d_id in ranked_docs:
        results.loc[count,'for_query'] = query_input
        results.loc[count,'tweet'] = df_tweets.loc[d_id, "full_text"]
        results.loc[count,'username'] = df_tweets.loc[d_id, "user"]['screen_name']
        results.loc[count,'date'] = df_tweets.loc[d_id, "created_at"]
        results.loc[count,'hashtags'] = [df_tweets.loc[0,'entities']['hashtags'][i]['text'] for i in range(len(df_tweets.loc[0,'entities']['hashtags']))]
        results.loc[count,'likes'] = df_tweets.loc[d_id, "favorite_count"]
        results.loc[count,'retweets'] = df_tweets.loc[d_id, "retweet_count"]
        results.loc[count,'cluster'] = df_tweets.loc[d_id, "cluster"]
        results.loc[count,'url'] = "https://twitter.com/twitter/statuses/"+str(df_tweets.loc[d_id, "id"])
        count +=1
    

    if scoring  == 3:
        results = diversify_correction(results,top)
        print("the diversity score obtained is ",diversify_score(results))
    else:
        print("the diversity score obtained is ",diversify_score(results[:top]))        
    
    return results[:top]

In [22]:
totalresults = pd.DataFrame()

In [41]:
results = userInteraction()
totalresults = totalresults.append(results)

Insert: 
1 if you want to use the inverted index 
2 if you want to use our score,
3 if you want to return a more diverse output
3

How many returned tweets you want?
10

Insert your query:
All lives matter

Doing the ranking.. 

Top 10 results out of 399 for the searched query:

the diversity score obtained is  0.4


In [42]:
totalresults.shape

(73, 9)

In [43]:
totalresults.to_csv("../other_outputs/rq2a-with.tsv", sep="\t")